## Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
import os
import glob
import torch
import wandb
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from torch import nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader

import torchvision

# os.environ["WANDB_API_KEY"] = ""

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# wandb.login()

## DataLoader

In [5]:
class RipDataset(Dataset):
    def __init__(self, rootdir=os.path.join(os.getcwd(), "VGG16_Training_Features"), view="frontal_view", split="train", train_split_percent=0.75, transform=None, merge_views=False, normalize_len=True, seq_len=400):
        super().__init__()

        self.rootdir = rootdir
        self.view = view
        self.transform = transform
        self.merge_views = merge_views
        self.train_split_percent = train_split_percent
        self.seq_len = seq_len
        self.normalize_len = normalize_len
        self.learnable_padding = nn.Parameter(torch.zeros(seq_len, 512))

        self.views = [view.split('/')[-1] for view in glob.glob(os.path.join(rootdir, '*'))]

        self.classes = [
            path.split('/')[-1]
            for path in glob.glob(os.path.join(self.rootdir, self.view, "*"))
        ]

        self.class_encoding = {
            classname: idx for idx, classname in enumerate(self.classes)
        }

        self.view_wise_paths = {
            view: glob.glob(
                os.path.join(self.rootdir, view, "*", '*.npy')
            )
            for view in self.views
        }

        self.classwise_paths = {
            classname: glob.glob(
                os.path.join(self.rootdir, self.view, classname, '*.npy')
            )
            for classname in self.classes
        }

        self.sample_class_map = {
            idx: self.class_encoding[path.split('/')[-2]]
            for idx, path in enumerate(glob.glob(os.path.join(rootdir, self.view, "*", "*.npy")))
        }

        if view not in self.views:
            raise ValueError("Invalid view name")

        if merge_views:
            self.total_paths_dict = {
                classname: [
                    {
                        view: self.view_wise_paths[view][idx]
                        for view in self.views
                    }
                    for idx, classval in self.sample_class_map.items()
                    if classval == self.class_encoding[classname]
                ]
                for classname in self.classes
            }
        else:
            self.total_paths_dict = {
                classname: glob.glob(
                    os.path.join(rootdir, view, classname, '*.npy')
                )
                for classname in self.classes
            }

        self.total_paths = [path_set for path_sets in self.total_paths_dict.values() for path_set in path_sets]
        random.shuffle(self.total_paths)

        self.train_len = int(len(self.total_paths) * self.train_split_percent)

        self.train_paths = self.total_paths[:self.train_len]
        self.valid_paths = self.total_paths[self.train_len:]

        if split == "train":
            self.paths = self.train_paths
        elif split == "valid":
            self.paths = self.valid_paths
        else:
            raise ValueError("Invalid split name, choose between `train` and `valid` only.")

    def __len__(self):
        return len(self.paths)

    def resize_sequence(self, features):
        if features.shape[0] < self.seq_len:
            diff = self.seq_len - features.shape[0]
            pad = torch.zeros(diff, features.shape[1], features.shape[2])  # Ensure pad has the same dimensions as features
            features = torch.cat((features, pad), dim=0)
            return features
        else:
            mid_point = features.shape[0] // 2
            window_size = self.seq_len // 2
            return features[int(mid_point - window_size): int(mid_point + window_size), :, :]


    def normalize_features(self, features):
        mean = features.mean()
        std = features.std()
        return (features - mean) / std

    def __getitem__(self, idx):
        if self.merge_views:
            path_list = [path for path in self.paths[idx].values()]
            features = [np.load(path) for path in path_list]
            features = np.concatenate(features, axis=-2)
            labels = self.class_encoding[path_list[0].split('/')[-2]]
            filenames = path_list[0].split('/')[-1].replace('.npy', '')
        else:
            path = self.paths[idx]
            features = np.load(path)
            labels = self.class_encoding[path.split('/')[-2]]
            filenames = path.split('/')[-1].replace('.npy', '')

        if self.transform:
            features = self.transform(features)

        features = torch.from_numpy(features)
        features = self.normalize_features(features)  # Normalize features
        labels = torch.tensor(labels, dtype=torch.long)  # Ensure label is a long tensor

        if self.normalize_len:
            features = self.resize_sequence(features)

        return {"features": features, "labels": labels, "filenames": filenames}

    def collate_fn(self, batch):
        features_list = [res["features"] for res in batch]
        labels_list = [res["labels"] for res in batch]
        filenames_list = [res["filenames"] for res in batch]

        features = torch.stack(features_list, dim=0)
        labels = torch.tensor(labels_list, dtype=torch.long)

        return {
            "features": features,
            "labels": labels,
            "filenames": filenames_list
        }

In [6]:
# Create the training and validation datasets
train_data_combined = RipDataset(rootdir='/kaggle/input/riptide-training/VGG16_Training_Features', view="frontal_view", split="train", merge_views=True, normalize_len=True, train_split_percent=1)
valid_data_combined = RipDataset(rootdir='/kaggle/input/riptide-val/VGG16_val_features', view="frontal_view", split="valid", merge_views=True, normalize_len=True, train_split_percent=0)

# Create DataLoader objects for the training and validation datasets
train_ds_combined = DataLoader(train_data_combined, batch_size=32, shuffle=True, collate_fn=train_data_combined.collate_fn)
valid_ds_combined = DataLoader(valid_data_combined, batch_size=32, shuffle=False, collate_fn=valid_data_combined.collate_fn)

In [7]:
# Create the training and validation datasets
train_data_frontal = RipDataset(rootdir='/kaggle/input/riptide-training/VGG16_Training_Features', view="frontal_view", split="train", merge_views=False, normalize_len=True, train_split_percent=1)
valid_data_frontal = RipDataset(rootdir='/kaggle/input/riptide-val/VGG16_val_features', view="frontal_view", split="valid", merge_views=False, normalize_len=True, train_split_percent=0)

# Create DataLoader objects for the training and validation datasets
train_ds_frontal = DataLoader(train_data_frontal, batch_size=32, shuffle=True, collate_fn=train_data_frontal.collate_fn)
valid_ds_frontal = DataLoader(valid_data_frontal, batch_size=32, shuffle=False, collate_fn=valid_data_frontal.collate_fn)

In [8]:
# Shape: [batch_size, seq_len, num_views, feature_dim]
# [32, 400, 3, 512]

# Shape: [batch_size]
# [32]

for i in train_ds_combined:
    print(i.keys())
    print(i['labels'].shape)
    print(i['features'].shape)
    break

dict_keys(['features', 'labels', 'filenames'])
torch.Size([32])
torch.Size([32, 400, 3, 512])


In [9]:
for i in valid_ds_combined:
    print(i.keys())
    print(i['labels'].shape)
    print(i['features'].shape)
    break

dict_keys(['features', 'labels', 'filenames'])
torch.Size([32])
torch.Size([32, 400, 3, 512])


In [10]:
for i in train_ds_frontal:
    print(i.keys())
    print(i['labels'].shape)
    print(i['features'].shape)
    break

dict_keys(['features', 'labels', 'filenames'])
torch.Size([32])
torch.Size([32, 400, 1, 512])


In [11]:
for i in valid_ds_frontal:
    print(i.keys())
    print(i['labels'].shape)
    print(i['features'].shape)
    break

dict_keys(['features', 'labels', 'filenames'])
torch.Size([32])
torch.Size([32, 400, 1, 512])


In [12]:
for i in valid_ds_frontal:
    print(i['labels'])
    break

tensor([4, 2, 4, 2, 5, 2, 2, 2, 1, 4, 4, 2, 4, 3, 4, 2, 1, 3, 3, 5, 2, 5, 1, 4,
        1, 2, 4, 2, 2, 4, 1, 4])


In [13]:
def check_class_labels(dataset):
    for class_name, label in dataset.class_encoding.items():
        print(f"Class name: {class_name}, Label: {label}")

# Check class names and labels for each dataset
print("Combined View Training Dataset:")
check_class_labels(train_data_combined)
print("\nCombined View Validation Dataset:")
check_class_labels(valid_data_combined)
print("\nFrontal View Training Dataset:")
check_class_labels(train_data_frontal)
print("\nFrontal View Validation Dataset:")
check_class_labels(valid_data_frontal)


Combined View Training Dataset:
Class name: Right Lane Change, Label: 0
Class name: Straight, Label: 1
Class name: Left Turn, Label: 2
Class name: Slow-Stop, Label: 3
Class name: Right Turn, Label: 4
Class name: Left Lane Change, Label: 5

Combined View Validation Dataset:
Class name: Right Lane Change, Label: 0
Class name: Straight, Label: 1
Class name: Left Turn, Label: 2
Class name: Slow-Stop, Label: 3
Class name: Right Turn, Label: 4
Class name: Left Lane Change, Label: 5

Frontal View Training Dataset:
Class name: Right Lane Change, Label: 0
Class name: Straight, Label: 1
Class name: Left Turn, Label: 2
Class name: Slow-Stop, Label: 3
Class name: Right Turn, Label: 4
Class name: Left Lane Change, Label: 5

Frontal View Validation Dataset:
Class name: Right Lane Change, Label: 0
Class name: Straight, Label: 1
Class name: Left Turn, Label: 2
Class name: Slow-Stop, Label: 3
Class name: Right Turn, Label: 4
Class name: Left Lane Change, Label: 5


## Metrics

In [14]:
def custom_accuracy(predictions, targets):
    """
    Compute the accuracy based on the provided formula.
    Args:
        predictions (Tensor): The predicted labels.
        targets (Tensor): The ground truth labels.
    Returns:
        float: The accuracy.
    """
    correct_predictions = (predictions == targets).sum().item()
    accuracy = correct_predictions / targets.size(0)
    return accuracy

In [22]:
def custom_f1_score(predictions, labels):
    tp = 0  # (true positive) correct prediction of the maneuver in a video
    fp = 0  # (false predictions) prediction is different than the actual performed maneuver
    fpp = 0 # (false positive prediction) a maneuver-action predicted, but the driver is driving straight
    mp = 0  # (missed prediction) a driving-straight predicted, but a maneuver is performed

    for i in range(len(predictions)):
        if predictions[i] == labels[i]:
            tp += 1
        else:
            fp += 1

            # a maneuver-action predicted, but the driver is driving straight
            if predictions[i] == 1:
                mp += 1

            # a driving-straight predicted, but a maneuver is performed
            if labels[i] == 1:
                fpp += 1

    if tp + fp + fpp == 0:
        precision = 0
        return 0
    else:
        precision = tp / (tp + fp + fpp)

    if tp + fp + mp == 0:
        recall = 0
        return 0
    else:
        recall = tp / (tp + fp + mp)

    if precision + recall == 0:
        f1 = 0
    else:
        f1 = 2 * precision * recall / (precision + recall)

    return f1

In [26]:
# checking for zero division error
predictions = torch.tensor([2, 3, 5, 6, 6, 6, 6, 6])
labels = torch.tensor([1, 1, 4, 0, 4, 3, 0, 2])

In [27]:
custom_f1_score(predictions, labels)

0

In [28]:
custom_accuracy(predictions, labels)

0.0

## Defining Models

In [30]:
class Model(nn.Module):
    def __init__(self, num_classes=6):
        super(Model, self).__init__()
        self.num_classes = num_classes

    def forward(self, x):
        return x

    def fit(self, train_loader, valid_loader, epochs, is_wandb=False, device='cuda', use_scheduler=False, save_name="model"):
        optimizer = optim.Adam(self.parameters(), lr=0.001, weight_decay=1e-5)
        criterion = nn.CrossEntropyLoss()
        scheduler = None

        if use_scheduler:
            scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

        for epoch in range(epochs):
            self.train()
            for batch_idx, data in enumerate(train_loader):
                features = data["features"]
                label = data["labels"]

                features = features.to(device)
                label = label.to(device)

                optimizer.zero_grad()
                output = self(features)
                loss = criterion(output, label)
                loss.backward()

                torch.nn.utils.clip_grad_norm_(self.parameters(), max_norm=1.0)

                optimizer.step()

                predictions = torch.argmax(output, dim=1)
                train_acc = custom_accuracy(predictions, label)
                train_f1 = custom_f1_score(predictions, label)

                if batch_idx % 100 == 0:
                    print(f"Epoch: {epoch+1} | Batch: {batch_idx} | CrossEntropyLoss: {loss.item():.3f} | Accuracy: {train_acc:.3f} | F1Score: {train_f1:.3f}")

            if scheduler:
                scheduler.step()

            self.eval()
            with torch.no_grad():
                val_f1_scores = []
                val_acc_scores = []
                for val_idx, val_data in enumerate(valid_loader):
                    val_features = val_data["features"]
                    val_label = val_data["labels"]

                    val_features = val_features.to(device)
                    val_label = val_label.to(device)

                    val_output = self(val_features)

                    val_predictions = torch.argmax(val_output, dim=1)
                    val_acc = custom_accuracy(val_predictions, val_label)
                    val_f1 = custom_f1_score(val_predictions, val_label)

                    val_f1_scores.append(val_f1)
                    val_acc_scores.append(val_acc)

                avg_val_f1_score = sum(val_f1_scores) / len(val_f1_scores)
                avg_val_acc_score = sum(val_acc_scores) / len(val_acc_scores)

                print(f"Validation -> Epoch: {epoch} | Accuracy: {avg_val_acc_score:.3f} | F1Score: {avg_val_f1_score:.3f}")

                if avg_val_acc_score > 0.90:
                    torch.save(self.state_dict(), f"{save_name}_val_acc_above_90.pt")
                    print(f"Model saved with validation accuracy above 90% at epoch {epoch}")

            if epoch == epochs - 1:
                torch.save(self.state_dict(), f"{save_name}_last_epoch.pt")
                print(f"Model saved at the last epoch {epoch}")


In [31]:
class CNN_LSTM_MULTI(Model):
    def __init__(self, num_classes=6, lstm_layers=2, hidden_size=64, dropout_rate=0.25):
        super(CNN_LSTM_MULTI, self).__init__(num_classes)
        self.hidden_size = hidden_size
        self.lstm_layers = lstm_layers

        self.conv1 = nn.Conv1d(in_channels=512, out_channels=64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.LeakyReLU()
        self.dropout = nn.Dropout(dropout_rate)

        self.lstm = nn.LSTM(64 * 3, hidden_size, batch_first=True, num_layers=lstm_layers, bidirectional=True, dropout=dropout_rate)
        self.fc = nn.Linear(hidden_size * 2, self.num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        batch_size, seq_len, num_views, feature_dim = x.size()
        # x: [32, 400, 3, 512]

        # Process each view separately
        features = []
        for view in range(num_views):
            view_x = x[:, :, view, :]
            view_x = view_x.transpose(1, 2)  # Transpose to [batch_size, feature_dim, seq_len]
            view_x = self.conv1(view_x)
            view_x = self.bn1(view_x)
            view_x = self.relu(view_x)
            view_x = self.dropout(view_x)
            view_x = view_x.transpose(1, 2)  # Transpose back to [batch_size, seq_len, feature_dim]
            features.append(view_x)

        # Concatenate features from all views
        x = torch.cat(features, dim=2)
        # x: [32, 400, 64 * 3]

        self.lstm.flatten_parameters()
        lstm_out, _ = self.lstm(x)
        # LSTM processing, output shape: [32, 400, hidden_size * 2]

        lstm_out = lstm_out[:, -1, :]
        # Get the output from the last time step, output shape: [32, hidden_size * 2]

        output = self.fc(lstm_out)
        # Fully connected layer, output shape: [32, num_classes]

        output = self.softmax(output)
        # Softmax, output shape: [32, num_classes]

        return output

In [32]:
class CNN_LSTM_FRONTAL(Model):
    def __init__(self, num_classes=6, lstm_layers=2, hidden_size=64, dropout_rate=0.25):
        super(CNN_LSTM_FRONTAL, self).__init__(num_classes)
        self.hidden_size = hidden_size
        self.lstm_layers = lstm_layers

        self.conv1 = nn.Conv1d(in_channels=512, out_channels=64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.LeakyReLU()
        self.dropout = nn.Dropout(dropout_rate)

        self.lstm = nn.LSTM(64, hidden_size, batch_first=True, num_layers=lstm_layers, bidirectional=True, dropout=dropout_rate)
        self.fc = nn.Linear(hidden_size * 2, self.num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        batch_size, seq_len, num_views, feature_dim = x.size()
        # x: [32, 230, 1, 512]

        x = x.squeeze(2)  # Remove the dimension of size 1
        # x: [32, 230, 512]

        x = x.transpose(1, 2)  # Transpose to [batch_size, feature_dim, seq_len]
        # x: [32, 512, 230]

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout(x)
        # x: [32, 64, 230]

        x = x.transpose(1, 2)  # Transpose back to [batch_size, seq_len, feature_dim]
        # x: [32, 230, 64]

        self.lstm.flatten_parameters()
        lstm_out, _ = self.lstm(x)
        # LSTM processing, output shape: [32, 230, hidden_size * 2]

        lstm_out = lstm_out[:, -1, :]
        # Get the output from the last time step, output shape: [32, hidden_size * 2]

        output = self.fc(lstm_out)
        # Fully connected layer, output shape: [32, num_classes]

        output = self.softmax(output)
        # Softmax, output shape: [32, num_classes]

        return output

## Training

In [33]:
param_grid = [
    {
        'lr': 0.001,
        'batch_size': 16,
        'epochs': 400,
        'lstm_layers': 2,
        'hidden_size': 128,
        'dropout_rate': 0.25,
        'use_scheduler': False
    }
    
]

In [34]:
# Training script for CNN_LSTM_MULTI
def train_cnn_lstm_multi_with_params(params, train_ds, valid_ds, device):
    save_name = f"cnn_lstm_multi_lr_{params['lr']}_dropout_{params['dropout_rate']}"
    
    # Initialize wandb
    wandb.init(project='cnn_lstm_multi_project', config=params)
    
    cnn_lstm_model = CNN_LSTM_MULTI(num_classes=6, lstm_layers=params['lstm_layers'], hidden_size=params['hidden_size'], dropout_rate=params['dropout_rate'])
    cnn_lstm_model = cnn_lstm_model.to(device)

    cnn_lstm_model.fit(
        train_loader=train_ds,
        valid_loader=valid_ds,
        epochs=params['epochs'],
        is_wandb=True,
        device=device,
        use_scheduler=params['use_scheduler'],
        save_name=save_name
    )

    # Finish the wandb run
    wandb.finish()
    
    return cnn_lstm_model

for params in param_grid:
    cnn_lstm_model = train_cnn_lstm_multi_with_params(params, train_ds_combined, valid_ds_combined, device)

    model_save_path = f"cnn_lstm_multi_model_final_lr_{params['lr']}_dropout_{params['dropout_rate']}.pt"
    torch.save(cnn_lstm_model.state_dict(), model_save_path)
    print(f"Model saved to {model_save_path}")

Epoch: 1 | Batch: 0 | CrossEntropyLoss: 1.793 | Accuracy: 0.062 | F1Score: 0.056
Validation -> Epoch: 0 | Accuracy: 0.393 | F1Score: 0.379
Epoch: 2 | Batch: 0 | CrossEntropyLoss: 1.598 | Accuracy: 0.438 | F1Score: 0.424
Validation -> Epoch: 1 | Accuracy: 0.393 | F1Score: 0.379
Epoch: 3 | Batch: 0 | CrossEntropyLoss: 1.579 | Accuracy: 0.375 | F1Score: 0.358
Validation -> Epoch: 2 | Accuracy: 0.402 | F1Score: 0.388
Epoch: 4 | Batch: 0 | CrossEntropyLoss: 1.617 | Accuracy: 0.281 | F1Score: 0.273
Validation -> Epoch: 3 | Accuracy: 0.393 | F1Score: 0.379
Epoch: 5 | Batch: 0 | CrossEntropyLoss: 1.691 | Accuracy: 0.219 | F1Score: 0.212
Validation -> Epoch: 4 | Accuracy: 0.393 | F1Score: 0.379
Epoch: 6 | Batch: 0 | CrossEntropyLoss: 1.654 | Accuracy: 0.312 | F1Score: 0.303
Validation -> Epoch: 5 | Accuracy: 0.393 | F1Score: 0.379
Epoch: 7 | Batch: 0 | CrossEntropyLoss: 1.599 | Accuracy: 0.406 | F1Score: 0.394
Validation -> Epoch: 6 | Accuracy: 0.442 | F1Score: 0.433
Epoch: 8 | Batch: 0 | Cross

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_acc,▁▃▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████████████
train_f1,▁▃▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████████████
train_loss,█▇▄▄▄▄▃▃▃▂▂▂▂▂▂▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▄▆▅▆▆▆▅▆▅▆▆▅▆▅▆▅▆▆▆▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▇▇▇▇█
val_f1,▁▄▆▅▆▆▆▅▆▅▆▆▅▆▅▆▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█
epoch,400
train_acc,0.9457
train_f1,0.94219
train_loss,1.09822
val_acc,0.71429


Model saved to cnn_lstm_multi_model_final_lr_0.001_dropout_0.25.pt


In [35]:
# Training script for CNN_LSTM_FRONTAL
def train_cnn_lstm_frontal_with_params(params, train_ds, valid_ds, device):
    save_name = f"cnn_lstm_frontal_lr_{params['lr']}_dropout_{params['dropout_rate']}"
    
    # Initialize wandb
    wandb.init(project='cnn_lstm_multi_project', config=params)
    
    cnn_lstm_model = CNN_LSTM_FRONTAL(num_classes=6, lstm_layers=params['lstm_layers'], hidden_size=params['hidden_size'], dropout_rate=params['dropout_rate'])
    cnn_lstm_model = cnn_lstm_model.to(device)

    cnn_lstm_model.fit(
        train_loader=train_ds,
        valid_loader=valid_ds,
        epochs=params['epochs'],
        is_wandb=True,
        device=device,
        use_scheduler=params['use_scheduler'],
        save_name=save_name
    )

    # Finish the wandb run
    wandb.finish()
    
    return cnn_lstm_model

for params in param_grid:
    cnn_lstm_model = train_cnn_lstm_frontal_with_params(params, train_ds_frontal, valid_ds_frontal, device)

    model_save_path = f"cnn_lstm_frontal_model_final_lr_{params['lr']}_dropout_{params['dropout_rate']}.pt"
    torch.save(cnn_lstm_model.state_dict(), model_save_path)
    print(f"Model saved to {model_save_path}")

Epoch: 1 | Batch: 0 | CrossEntropyLoss: 1.794 | Accuracy: 0.094 | F1Score: 0.086
Validation -> Epoch: 0 | Accuracy: 0.366 | F1Score: 0.352
Epoch: 2 | Batch: 0 | CrossEntropyLoss: 1.664 | Accuracy: 0.250 | F1Score: 0.239
Validation -> Epoch: 1 | Accuracy: 0.366 | F1Score: 0.352
Epoch: 3 | Batch: 0 | CrossEntropyLoss: 1.590 | Accuracy: 0.500 | F1Score: 0.485
Validation -> Epoch: 2 | Accuracy: 0.478 | F1Score: 0.471
Epoch: 4 | Batch: 0 | CrossEntropyLoss: 1.938 | Accuracy: 0.094 | F1Score: 0.065
Validation -> Epoch: 3 | Accuracy: 0.357 | F1Score: 0.343
Epoch: 5 | Batch: 0 | CrossEntropyLoss: 1.673 | Accuracy: 0.219 | F1Score: 0.212
Validation -> Epoch: 4 | Accuracy: 0.384 | F1Score: 0.372
Epoch: 6 | Batch: 0 | CrossEntropyLoss: 1.483 | Accuracy: 0.531 | F1Score: 0.531
Validation -> Epoch: 5 | Accuracy: 0.482 | F1Score: 0.470
Epoch: 7 | Batch: 0 | CrossEntropyLoss: 1.518 | Accuracy: 0.531 | F1Score: 0.515
Validation -> Epoch: 6 | Accuracy: 0.384 | F1Score: 0.372
Epoch: 8 | Batch: 0 | Cross

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_acc,▁▂▂▂▅▆▆▆▆▆▆▆▇▇▇▆▇▇▇▇▇▇▇▇▇███▇██████▇████
train_f1,▁▂▂▂▅▅▅▆▆▆▆▆▇▇▇▆▇▇▇▇▇▇▇▇▇███▇██████▇████
train_loss,█▇▇▇▅▄▄▄▄▃▃▃▂▂▂▃▂▂▂▂▂▂▂▂▂▁▁▂▂▁▁▁▁▁▂▂▁▁▁▁
val_acc,▁▂▁▂▄▇▆▇▆▆▇▆▆▆█▆▇▇▇▇▇▇▇▆▇▇▆█▇▇▇▇▇▇▆▆▇▇██
val_f1,▁▂▁▂▄▇▅▇▆▆▇▆▆▅█▆▇▇▇▇▇▇▇▆██▆█▇▇█▇▇▇▆▆▇▇██
epoch,400
train_acc,0.91484
train_f1,0.9088
train_loss,1.12947
val_acc,0.66964


Model saved to cnn_lstm_frontal_model_final_lr_0.001_dropout_0.25.pt
